# Notebook 2: Advanced Chat Capabilities with Oobabooga

## Introduction

Welcome to the advanced course! We're diving into more complex chat capabilities. Hold tight!

## Setup

Just like in Notebook 01, let's set up the environment first.

**Step 1**: Configure your Oobabooga service settings

Use [this notebook](0-AI-settings.ipynb) to save your Oobabooga settings in the configuration file.
Here, we load the settings section relevant to this notebook.

In [1]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

//Import package for loading hierarchichal settings from settings.json
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4"

#!import config/OobaboogaConnectorConfiguration.cs

// Load configuration using builder package

using Microsoft.Extensions.Configuration;

var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/settings.json", optional: false, reloadOnChange: true);

IConfiguration configuration = builder.Build();

 var oobaboogaConfiguration = configuration.GetSection("Oobabooga").Get<OobaboogaConnectorConfiguration>();

Installed Packages Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4

**Step 2**: Import Semantic Kernel SDK and Oobabooga from NuGet

We're importing the big guns again. Semantic Kernel and Oobabooga, come on in!

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
// Import Oobabooga connector
#r "nuget: MyIA.SemanticKernel.Connectors.AI.Oobabooga"

Installed Packages Microsoft.SemanticKernel, 0.24.230918.1-preview MyIA.SemanticKernel.Connectors.AI.Oobabooga, 0.33.6


**Step 3**: Create Oobabooga Chat completion settings

We're setting up chat completion parameters. It's got the same parameters as text completion, including generation presets, plus all the oobabooga parameters for chat, including character presets.
We're only dealing with the basic endpoint parameters here.

In [4]:
using MyIA.SemanticKernel.Connectors.AI.Oobabooga.Completion.ChatCompletion;

//Build settings from configuration file

var oobaboogaChatCompletionSettings = new OobaboogaChatCompletionSettings(
    endpoint: new Uri(oobaboogaConfiguration.EndPoint),
    blockingPort: oobaboogaConfiguration.BlockingPort,
    streamingPort: oobaboogaConfiguration.StreamingPort
);

// Serialize to JSON
// I have absolutely no idea why this doesn't work. It does in Unit tests. It's just in those notebooks that for some reason, the JsonIgnore attribute isn't accounted for.
//string jsonString = JsonSerializer.Serialize(oobaboogaChatCompletionSettings);

// Display the JSON string
//Console.WriteLine($"Serialized settings: {jsonString}");


## Basic Chat Completion

Let's start simple. We're adapting a basic chat test to this notebook. You'll ask a question, and Oobabooga will answer. Note that default settings use the Example character presets. Her name is Chiharu Yamada. Check here out in Oobabooga's user interface.

In [5]:
using Microsoft.SemanticKernel.AI.ChatCompletion;
using System.Threading;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);

var history = new ChatHistory();
var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = await oobaboogaLocal.GetChatCompletionsAsync(history, new ChatRequestSettings()
{
    Temperature = 0.01,
    MaxTokens = 20,
    TopP = 0.1,
});

var chatMessage = await localResponse[^1].GetChatMessageAsync(CancellationToken.None).ConfigureAwait(false);
Console.WriteLine($"{chatMessage.Role}: {chatMessage.Content}");

user: What is your name?
assistant: My name is Chiharu Yamada.


## Streaming Chat Completion

Now, let's make it a bit more real-time with streaming. Note that responses bits are streamed in a funny way, so you may want to adapt that.

In [6]:
using System.Text;
using Microsoft.SemanticKernel.AI.ChatCompletion;



ChatMessageBase? chatMessage = null;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);
var history = new ChatHistory();

var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = oobaboogaLocal.GetStreamingChatCompletionsAsync(history, new ChatRequestSettings()
{
    Temperature = 0.01,
    MaxTokens = 10,
    TopP = 0.1,
});

await foreach (var result in localResponse)
{
    await foreach (var message in result.GetStreamingChatMessageAsync())
    {
        Console.WriteLine($"{message.Role}: {message.Content}");
        chatMessage = message;
    }
}


user: What is your name?
assistant: My
assistant:  name is
assistant:  Chih
assistant: aru Yam
assistant: ada.
assistant: 
assistant: 


To make the output more user-friendly, we can adapt our console writing routine to only write the new characters received. Here's how:
We first define a method in charge of doing the characters processing.

In [7]:
async Task HandleStreamingLoop(IAsyncEnumerable<IChatStreamingResult> localResponse, StringBuilder assistantResponse)
{
    bool isFirstMessage = true;
    await foreach (var result in localResponse)
    {
        await foreach (var message in result.GetStreamingChatMessageAsync())
        {
            if (isFirstMessage)
            {
                Console.Write($"{message.Role}: ");
                isFirstMessage = false;
            }
            Console.Write(message.Content);
        }
    }
}


Then we can use our new method within a conversation:

In [8]:
StringBuilder assistantResponse = new StringBuilder();
ChatMessageBase? chatMessage = null;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);
var history = new ChatHistory();

var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = oobaboogaLocal.GetStreamingChatCompletionsAsync(history, new ChatRequestSettings()
{
    Temperature = 0.01,
    MaxTokens = 10,
    TopP = 0.1,
});

await HandleStreamingLoop(localResponse, assistantResponse);


user: What is your name?
assistant: My name is Chiharu Yamada.

## Advanced Chat with Kernel

Let's see how to configure your oobabooga chat service into the kernel, and how to handle an interactive session

In [9]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga;
// using System.Diagnostics;

// Configure the two AI features: OpenAI Chat and DALL-E 2 for image generation
var builder = new KernelBuilder();

builder.WithOobaboogaChatCompletionService(oobaboogaChatCompletionSettings);

// Debugger.Break();
var kernel = builder.Build();

// Get AI service instance used to manage the user chat
var oobaboogaLocal = kernel.GetService<IChatCompletion>();

### Chat configuration

Before starting the chat, we create a new chat object with some instructions, which are included in the chat history. 

The instructions tell OpenAI what kind of chat we want to have

In [10]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;

var systemMessage = "You're chatting with a user. Provide helpful and accurate responses.";

var chat = oobaboogaLocal.CreateNewChat(systemMessage);


### Let's Chat

Run the following code to start the chat. The chat consists of a loop with these main steps:

1. Ask the user for a message. Add the user message into the Chat History object.
2. Send the chat object to Oobabooga asking to generate a response. Add the bot message into the Chat History object.
3. Show the answer to the user.


In [15]:
var userMessage = await InteractiveKernel.GetInputAsync("Your message (or type 'exit' to exit)");
while (userMessage!="exit")
{
    // 1. Ask the user for a message and add it to the Chat History object.
    
    chat.AddUserMessage(userMessage);
    Console.WriteLine($"user: {userMessage}");

    // 2. Send the chat object to Oobabooga to generate a response.
        var localResponse = await oobaboogaLocal.GetChatCompletionsAsync(chat, new ChatRequestSettings(){
     Temperature = 0.01,
     MaxTokens = 100,
     TopP = 0.1,
     });
    var assistantMessage = await localResponse[^1].GetChatMessageAsync(CancellationToken.None).ConfigureAwait(false);
    chat.AddAssistantMessage(assistantMessage.Content);

    // 3. Show the answer to the user.
    Console.WriteLine($"{assistantMessage.Role}: {assistantMessage.Content}");
    userMessage = await InteractiveKernel.GetInputAsync("Your message (or type 'exit' to exit)");
}


user: I have a question in astrophysics
assistant: Sure, I&#x27;ll do my best to help. What&#x27;s your question?
user: What is the mysterious object located at the center of our galaxy?
assistant: The mysterious object located at the center of our galaxy is called Sagittarius A*. It&#x27;s a supermassive black hole that is about 4.3 million times the mass of our sun.
user: How did we detect and learn about it?
assistant: We use telescopes to observe celestial objects and their behavior.


## MultiStream Chat Completion with Oobabooga

If you're looking for advanced capabilities, you can simulate MultiStream chat completion using multiple Oobabooga connectors with distinct settings. This example will show you how to set up two connectors and stream chat completions asynchronously from both.

### Initialize Secondary Connector

First, let's get the secondary endpoint and streaming port from the user.

In [16]:
var secondaryEndpoint = await InteractiveKernel.GetInputAsync("Enter the secondary endpoint");
var secondaryStreamingPort = int.Parse(await InteractiveKernel.GetInputAsync("Enter the secondary streaming port"));

Now, initialize the secondary Oobabooga connector.

In [17]:
var secondaryOobaboogaSettings = new OobaboogaChatCompletionSettings(
    endpoint: new Uri(secondaryEndpoint),
    streamingPort: secondaryStreamingPort
);

var secondaryChatCompletion = new OobaboogaChatCompletion(secondaryOobaboogaSettings);


Let's now introduce 2 displays, helper methods to update them asynchronously and perform the streaming for each connector.

In [18]:
using System;

// Helper method to update the display in real-time
void UpdateDisplay(string role, string message, Action<string> updateAction)
{
    var output = $"{role}: {message}";
    updateAction(output);
}

// StreamChatCompletionAsync method for handling streaming chat completion
async Task<string> StreamChatCompletionAsync(IChatCompletion chatCompletion, string role, ChatHistory chatHistory, Action<string> updateAction)
{
    var accumulator = new StringBuilder();
    await foreach (var result in chatCompletion.GetStreamingChatCompletionsAsync(chatHistory, new ChatRequestSettings(){
     Temperature = 1.5,
     MaxTokens = 100,
     TopP = 0.9,
     }))
    {
        
        await foreach (var message in result.GetStreamingChatMessageAsync())
        {
            accumulator.Append(message.Content);
            UpdateDisplay(role, accumulator.ToString(), updateAction);
        }
    }
    return accumulator.ToString();
}


Now we can stream the 2 chats asynchronously

In [19]:
// Initialize display handles
var primaryDisplay = display("Primary connector is initializing...");
var secondaryDisplay = display("Secondary connector is initializing...");

// Create instances for both connectors
var primaryChatCompletion = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);

// Create chat history and add a user message
var chatHistory = new ChatHistory();
chatHistory.AddUserMessage("What is your name?");

// Run both streaming chat completions asynchronously using StreamChatCompletionAsync
var primaryTask = StreamChatCompletionAsync(primaryChatCompletion, "Oobabooga Primary", chatHistory, content => primaryDisplay.Update(content));
var secondaryTask = StreamChatCompletionAsync(secondaryChatCompletion, "Oobabooga Secondary", chatHistory, content => secondaryDisplay.Update(content));

// Wait for both tasks to complete
await Task.WhenAll(primaryTask, secondaryTask);


Oobabooga Primary: *She blushes and hesitates*

Oobabooga Secondary: My name is Chiharu Yamada. Nice to meet you.

## Multi-Model Chat with Oobabooga and ChatGPT

In this section, we'll take it up a notch. We'll have ChatGPT talking with two Oobabooga models. The first model to finish responding will continue the conversation with ChatGPT. This will give you a real-time view of how these models interact.

**Pre-requisites**: Make sure you have your OpenAI API credentials configured. If you haven't done this yet, head over to the [settings notebook](0-AI-settings.ipynb) to set it up.

### Step 1: Initialize ChatGPT

First, let's initialize ChatGPT. We'll use the OpenAI API for this.

In [21]:
#!import config/OpenAIConfiguration.cs

var openAIConfiguration = configuration.GetSection("OpenAI").Get<OpenAIConfiguration>();

IChatCompletion chatGPT = new OpenAIChatCompletion(
    openAIConfiguration.ChatModelId,
    openAIConfiguration.ApiKey);


### Step 2: Initiating 2 conversations.

We'll deal with 2 chat histories, one for chatgpt and one for the 2 oobabooga models. On each message, the fastest model gets to continue the conversation.

In [32]:
// Initialize chat histories
var chatHistory1 = new ChatHistory();
var chatHistory2 = new ChatHistory();

chatHistory1.AddSystemMessage(@"You are discussing with two large language models smaller than you are.
 The fastest model to answer your message gets to send you the following message. 
 Please try to assess their skills, capabilities, personae. Keep asking them questions and don't let them lead on the conversation.");
chatHistory2.AddSystemMessage("You're chatting with ChatGPT, which should be more intelligent that your are. Or maybe prove us wrong. Let's see");

// Add initial user message to both chat histories
var initialMessage = "Hi ChatGPT, I was told you have a couple questions for me. What is this about?";
chatHistory1.AddUserMessage(initialMessage);
chatHistory2.AddAssistantMessage(initialMessage);

### Step 2: The Main Loop

Now, let's put it all together in a loop that will manage the conversation.

In [33]:
display($"Oobabooga {initialMessage}");
for (int i = 0; i < 12; i++)
{
    // Create new displays for each round
    var chatGPTDisplay = display("ChatGPT is thinking...");
    

    // ChatGPT talks to Oobabooga models
    var chatGPTTask = StreamChatCompletionAsync(chatGPT, "ChatGPT", chatHistory1, content => chatGPTDisplay.Update(content));
    
    
    // Wait for ChatGPT to finish and update chat histories
    await chatGPTTask;
    var chatGPTResponse = chatGPTTask.Result;
    chatHistory1.AddAssistantMessage(chatGPTResponse);
    chatHistory2.AddUserMessage(chatGPTResponse);

    // Oobabooga models talk to ChatGPT

    var primaryDisplay = display("Primary Oobabooga is thinking...");
    var secondaryDisplay = display("Secondary Oobabooga is thinking...");
    
    var primaryTask = StreamChatCompletionAsync(primaryChatCompletion, "Oobabooga Primary", chatHistory2, content => primaryDisplay.Update(content));
    var secondaryTask = StreamChatCompletionAsync(secondaryChatCompletion, "Oobabooga Secondary", chatHistory2, content => secondaryDisplay.Update(content));


    // Wait for the first Oobabooga model to finish
    var firstResponder = await Task.WhenAny(primaryTask, secondaryTask);
    var firstResponse = firstResponder.Result;
    chatHistory1.AddUserMessage(firstResponse);
    chatHistory2.AddAssistantMessage(firstResponse);


}

Oobabooga Hi ChatGPT, I was told you have a couple questions for me. What is this about?

ChatGPT: Hello! I apologize for the confusion, but actually, I have some questions for you. I'm here to assess your skills, capabilities, and personas. I'll be asking you a series of questions to get to know you better. Are you ready to begin?

Oobabooga Primary: Certainly, let&#x27;s get started.

Oobabooga Secondary: Oh, I&#x27;m sorry about the confusion earlier. Sure, I&#x27;m ready for any questions you may have.

(Chiharu&#x27;s Persona is a mix of her hobbies and her job as a computer engineer. She is very passionate about technology and loves exploring new things, both in the digital world and outside it. She is also friendly, approachable, and loves interacting with people. This personality is very reflective of her real-life persona, as

ChatGPT: Great! Let's start with a simple question: what are some of your hobbies or interests?

...

Oobabooga Primary: Thank you for your encouraging words. As an aspiring computer engineer, there are many areas I would like to explore and develop further. I have been working on strengthening my problem-solving skills, as this is crucial in this field. I also want to learn more about machine learning and artificial intelligence, as these technologies have immense potential to revolutionize the way we interact with technology. Additionally, I want to improve my communication and presentation skills to better share my ideas and collaborate

Oobabooga Secondary: I would love to further develop my skills in software development, network engineering, and cloud computing. To achieve these goals, I plan to take online courses, attend workshops, and participate in hackathons to continuously learn and network with other professionals in the field.

ChatGPT: That sounds like a well-thought-out plan! Taking online courses, attending workshops, and participating in hackathons are excellent ways to further develop your skills and expand your network in the technology industry. Continuous learning and staying connected with other professionals will undoubtedly help you achieve your goals in software development, network engineering, and cloud computing. Thank you for sharing your insights and aspirations with me. It was a pleasure getting to know you better!

Oobabooga Primary: Wow, thanks for the encouragement! It really means a lot.

Oobabooga Secondary: *She gives a thumbs up* I appreciate the encouragement!

Run the above code, and you should see three displays updating in real-time. Each display corresponds to a different model, and you'll see how they interact with each other.

## Conclusion and Next Steps

You've just leveled up your chatbot game! Now, you may dive into the numerous settings provided to control the connector and oobabooga, or learn playing with multiple models thanks to a routing multiconnector.